# Notebook to test new functions

In [1]:
nest_dir = "neural_encoding_simulation_toolkit" 
test_images = "tutorial_images"

---

# Prepare all images

In [2]:
from tqdm import tqdm
from torchvision import transforms as trn
import numpy as np
import os
from PIL import Image

images_dir = os.path.join(test_images)
images_list = os.listdir(images_dir)
images_list.sort()
    
images = []
for img in tqdm(images_list):
    img_dir = os.path.join(images_dir, img)
    img = Image.open(img_dir).convert('RGB')
    # Center crop the images to square format, and resize them
    transform = trn.Compose([
        trn.CenterCrop(min(img.size)),
        trn.Resize((227,227))
    ])
    img = transform(img)
    img = np.asarray(img)
    img = img.transpose(2,0,1)
    images.append(img)
images = np.asarray(images)

# Print the images dimensions
print('\n\nImages shape:')
print(images.shape)
print('(Batch size × 3 RGB Channels x Width x Height)')

100%|██████████| 100/100 [00:00<00:00, 334.11it/s]




Images shape:
(100, 3, 227, 227)
(Batch size × 3 RGB Channels x Width x Height)


---

# How to run the code in general:

In [3]:
from nest import NEST

nest = NEST(nest_dir=nest_dir)

# List all available models and their versions
available_models = nest.list_models()
print(f"Available models: {available_models}")

# See what modalities are available
modalities = nest.which_modalities()
print(f"Available modalities: {modalities}")

Available models: {'fmri_nsd_fwrf': ['1.0.0'], 'eeg_things_eeg_2_vit_b_32': ['1.0.0']}
Available modalities: ['fmri', 'eeg']


In [4]:
# Load Encoding Model
fwrf_model = nest.get_encoding_model("fmri_nsd_fwrf", 
                                     subject=1, 
                                     roi="V1", 
                                     version="latest") # or '1.0.0', or whichever version will be available


eeg_model = nest.get_encoding_model("eeg_things_eeg_2_vit_b_32", 
                                    subject=1)

fwrf_silico = nest.encode(fwrf_model, images)
eeg_silico, eeg_metadata = nest.encode(eeg_model, images,return_metadata=True)

Model loaded on cpu for subject 1, ROI V1
Model loaded on cpu for subject 1


Encoding EEG responses: 100%|██████████| 1/1 [00:01<00:00,  1.72s/it, Encoded images=100, Total images=100]


In [5]:
print(fwrf_silico.shape)
print(eeg_silico.shape)
print(list(eeg_metadata.keys()))

(100, 1350)
(100, 4, 63, 140)
['eeg', 'encoding_models']


---

# fwRF Model in more detail

In [6]:
from nest import NEST
nest = NEST(nest_dir=nest_dir)

# Choose a model of interest
model_id = "fmri_nsd_fwrf"  # Example model ID

# Get comprehensive model information
model_info = nest.describe(model_id)


🧠 Model: fmri_nsd_fwrf
------------------------------------------------------------
Modality: fmri
Dataset: NSD
Features: x
Repeats: x
Subject level: x

📌 Supported Parameters:

• subject (int, required)
  ↳ Subject ID from the NSD dataset
  ↳ Valid values: [1, 2, 3, 4, 5, 6, 7, 8]
  ↳ Example: 1

• roi (str, required)
  ↳ Region of Interest (ROI) for voxel prediction
  ↳ Valid values: ['V1', 'V2', 'V3', 'hV4', 'EBA', 'FBA-2', 'OFA', 'FFA-1', 'FFA-2', 'PPA', 'RSC', 'OPA', 'OWFA', 'VWFA-1', 'VWFA-2', 'mfs-words', 'early', 'midventral', 'midlateral', 'midparietal', 'parietal', 'lateral', 'ventral']
  ↳ Example: V1

• nest_dir (str, optional)
  ↳ Root directory of the NEST repository (optional if default paths are set)
  ↳ Example: ./

📦 Example Usage:

from nest import NEST

nest = NEST("/path/to/nest_dir")
model = nest.get_encoding_model("fmri_nsd_fwrf", subject=1, roi='V1', nest_dir='./')

# Generate responses (assuming stimulus is a numpy array)
responses = model.generate_response(sti

In [8]:
# Load Encoding Model
fwrf_model = nest.get_encoding_model("fmri_nsd_fwrf", 
                                     subject=1, 
                                     roi="V1", 
                                     version="latest") # or '1.0.0', or whichever version will be available

Model loaded on cpu for subject 1, ROI V1


In [9]:
# Encode images
nest.encode(fwrf_model, images)

Encoding fMRI responses:   0%|          | 0/1 [00:00<?, ?it/s, Encoded images=0, Total images=100]

Encoding fMRI responses: 100%|██████████| 1/1 [00:00<00:00,  1.31it/s, Encoded images=100, Total images=100]


array([[-0.055599  ,  0.1633554 ,  0.0943293 , ...,  0.01447097,
         0.15747474,  0.01369774],
       [ 0.01819719,  0.07435445,  0.04977052, ..., -0.01315978,
         0.10419706,  0.03225161],
       [ 0.10229094,  0.21433882,  0.13944887, ...,  0.01901415,
         0.14813517,  0.00760403],
       ...,
       [ 0.04800354, -0.14399454, -0.28155515, ..., -0.02397447,
        -0.08016404,  0.00045429],
       [ 0.04686468,  0.07014671,  0.00538955, ..., -0.02637868,
         0.10473584,  0.02725478],
       [ 0.11645083,  0.02392432, -0.04246235, ..., -0.07162039,
        -0.0095483 , -0.01467158]], dtype=float32)

In [10]:
# Encode images and return metadata
nest.encode(fwrf_model, images, return_metadata=True)

Encoding fMRI responses:   0%|          | 0/1 [00:00<?, ?it/s, Encoded images=0, Total images=100]

Encoding fMRI responses: 100%|██████████| 1/1 [00:00<00:00,  1.31it/s, Encoded images=100, Total images=100]


(array([[-0.055599  ,  0.1633554 ,  0.0943293 , ...,  0.01447097,
          0.15747474,  0.01369774],
        [ 0.01819719,  0.07435445,  0.04977052, ..., -0.01315978,
          0.10419706,  0.03225161],
        [ 0.10229094,  0.21433882,  0.13944887, ...,  0.01901415,
          0.14813517,  0.00760403],
        ...,
        [ 0.04800354, -0.14399454, -0.28155515, ..., -0.02397447,
         -0.08016404,  0.00045429],
        [ 0.04686468,  0.07014671,  0.00538955, ..., -0.02637868,
          0.10473584,  0.02725478],
        [ 0.11645083,  0.02392432, -0.04246235, ..., -0.07162039,
         -0.0095483 , -0.01467158]], dtype=float32),
 {'fmri': {'ncsnr': array([0.10146447, 0.30089897, 0.2398103 , ..., 0.04201289, 0.22930188,
          0.06711598]),
   'roi_mask_volume': array([[[False, False, False, ..., False, False, False],
           [False, False, False, ..., False, False, False],
           [False, False, False, ..., False, False, False],
           ...,
           [False, False, F

In [11]:
# Just return metadata of model
fwrf_model.get_metadata()

{'fmri': {'ncsnr': array([0.10146447, 0.30089897, 0.2398103 , ..., 0.04201289, 0.22930188,
         0.06711598]),
  'roi_mask_volume': array([[[False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False],
          ...,
          [False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False]],
  
         [[False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False],
          ...,
          [False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False]],
  
         [[False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False],
   

--- 

# Testing EEG Model

In [14]:
from nest import NEST
nest = NEST(nest_dir=nest_dir)


eeg_model = nest.get_encoding_model("eeg_things_eeg_2_vit_b_32", 
                                    subject=1)

Model loaded on cpu for subject 1


In [15]:
nest.encode(eeg_model, images,return_metadata=True)

Encoding EEG responses:   0%|          | 0/1 [00:00<?, ?it/s]

Encoding EEG responses: 100%|██████████| 1/1 [00:01<00:00,  1.73s/it, Encoded images=100, Total images=100]


(array([[[[ 1.24397352e-01,  7.15943575e-02, -3.34081054e-03, ...,
            1.98921803e-02, -5.35618514e-04,  2.01637093e-02],
          [-3.15451100e-02,  4.23475057e-02,  1.02535307e-01, ...,
            9.39431414e-02,  2.16076255e-01,  2.00244278e-01],
          [-1.09511185e-02, -2.08470270e-01, -3.59001279e-01, ...,
           -8.20529461e-02, -6.36036918e-02, -1.31917879e-01],
          ...,
          [-2.55278051e-02, -9.43882950e-03,  4.79462147e-02, ...,
            1.25528991e-01,  1.35852039e-01,  7.83592612e-02],
          [-1.39562547e-01, -5.55058941e-02,  2.38118283e-02, ...,
            9.01250839e-02,  1.67249560e-01,  8.69320780e-02],
          [-6.74541444e-02, -1.07088499e-01, -1.07773006e-01, ...,
           -1.29966959e-01, -9.95634496e-02, -8.21912885e-02]],
 
         [[ 7.96499774e-02, -9.57980528e-02, -1.29369929e-01, ...,
           -1.10400274e-01, -1.01401001e-01, -6.63073510e-02],
          [-1.31157815e-01,  1.05764205e-02,  4.61376347e-02, ...,
     

---

### You can describe a model in two ways

In [16]:
nest.describe("eeg_things_eeg_2_vit_b_32")

🧠 Model: eeg_things_eeg_2_vit_b_32
------------------------------------------------------------
Modality: eeg
Dataset: things_eeg_2
Features: vision_transformer
Repeats: multi
Subject level: x

📌 Supported Parameters:

• subject (int, required)
  ↳ Subject ID from the things_eeg_2 dataset
  ↳ Valid values: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
  ↳ Example: 1

• nest_dir (str, optional)
  ↳ Root directory of the NEST repository (optional if default paths are set)
  ↳ Example: ./

📦 Example Usage:

from nest import NEST

nest = NEST("/path/to/nest_dir")
model = nest.get_encoding_model("eeg_things_eeg_2_vit_b_32", subject=1, nest_dir='./')

# Generate responses (assuming stimulus is a numpy array)
responses = model.generate_response(stimulus)



{'model_id': 'eeg_things_eeg_2_vit_b_32',
 'metadata': {'modality': 'eeg',
  'dataset': 'things_eeg_2',
  'features': 'vision_transformer',
  'repeats': 'multi',
  'subject_level': 'x'},
 'supported_parameters': {'subject': {'type': 'int',
   'required': True,
   'valid_values': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
   'example': 1,
   'description': 'Subject ID from the things_eeg_2 dataset'},
  'nest_dir': {'type': 'str',
   'required': False,
   'example': './',
   'description': 'Root directory of the NEST repository (optional if default paths are set)'}},
 'example_usage': 'from nest import NEST\n\nnest = NEST("/path/to/nest_dir")\nmodel = nest.get_encoding_model("eeg_things_eeg_2_vit_b_32", subject=1, nest_dir=\'./\')\n\n# Generate responses (assuming stimulus is a numpy array)\nresponses = model.generate_response(stimulus)'}

In [17]:
eeg_model.describe()

🧠 Model: eeg_things_eeg_2_vit_b_32
------------------------------------------------------------
Modality: eeg
Dataset: things_eeg_2
Features: vision_transformer
Repeats: multi
Subject level: x

📌 Supported Parameters:

• subject (int, required)
  ↳ Subject ID from the things_eeg_2 dataset
  ↳ Valid values: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
  ↳ Example: 1

• nest_dir (str, optional)
  ↳ Root directory of the NEST repository (optional if default paths are set)
  ↳ Example: ./

📦 Example Usage:

from nest import NEST

nest = NEST("/path/to/nest_dir")
model = nest.get_encoding_model("eeg_things_eeg_2_vit_b_32", subject=1, nest_dir='./')

# Generate responses (assuming stimulus is a numpy array)
responses = model.generate_response(stimulus)



{'model_id': 'eeg_things_eeg_2_vit_b_32',
 'metadata': {'modality': 'eeg',
  'dataset': 'things_eeg_2',
  'features': 'vision_transformer',
  'repeats': 'multi',
  'subject_level': 'x'},
 'supported_parameters': {'subject': {'type': 'int',
   'required': True,
   'valid_values': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
   'example': 1,
   'description': 'Subject ID from the things_eeg_2 dataset'},
  'nest_dir': {'type': 'str',
   'required': False,
   'example': './',
   'description': 'Root directory of the NEST repository (optional if default paths are set)'}},
 'example_usage': 'from nest import NEST\n\nnest = NEST("/path/to/nest_dir")\nmodel = nest.get_encoding_model("eeg_things_eeg_2_vit_b_32", subject=1, nest_dir=\'./\')\n\n# Generate responses (assuming stimulus is a numpy array)\nresponses = model.generate_response(stimulus)'}